In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen
from collections import defaultdict
from sklearn import linear_model

# import working with data libraries
import pandas as pd
import numpy as np

# import seaborn
import seaborn as sns

# import matplotlib
import matplotlib.pyplot as plt # Typical way of import MPL
import matplotlib as mpl # This line is used less frequently

In [2]:
### load the review data

review_data = []
with gzip.open('Software.json.gz') as f:
    for l in f:
        review_data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of reviews
print(len(review_data))

# first row of the list
print(review_data[10])

459436
{'overall': 2.0, 'vote': '3', 'verified': True, 'reviewTime': '05 3, 2009', 'reviewerID': 'A1CUJ6ZT82H7LS', 'asin': '0132147556', 'style': {'Format:': ' Paperback'}, 'reviewerName': 'Brian', 'reviewText': "Maybe it's just me (I have no marketing background but desperately want to learn for my start-up) but I cannot get hardly anything out of this text. I have tried very hard to tread through the writing and learn something useful but chapter after chapter seems to be the exact same thing... overly wordy, rambling & unnecessarily academic writing with no comprehensible message behind it, no strategies I can apply to my own business and no organization that I can make sense of. Feels almost like I'm reading a paper a college student BSed their way through with a whole bunch of long words and cryptic sentences in order to sound impressive. What a rip-off. I paid $70 for this and it has been the most useless book I've paid for in my research process (and I've read 16 other books thu

In [3]:
# convert list into pandas dataframe

all_products = {set}
df_review = pd.DataFrame.from_dict(review_data)

#filter out the non-verified reviews, and leave only the columns we want
# df_review = df_review[df_review['verified']==True][["overall", "reviewTime", "asin", "reviewText", "summary"]]
df_review = df_review[["overall", "reviewTime", "asin", "reviewText", "summary"]]
df_review = df_review.rename(columns={"asin": "product_id", "overall": "rating"})
print(len(df_review))
df_review.head()

459436


,rating,reviewTime,product_id,reviewText,summary
0,4.0,"03 11, 2014",0077613252,The materials arrived early and were in excell...,Material Great
1,4.0,"02 23, 2014",0077613252,I am really enjoying this book with the worksh...,Health
2,1.0,"02 17, 2014",0077613252,"IF YOU ARE TAKING THIS CLASS DON""T WASTE YOUR ...",ARE YOU KIDING ME?
3,3.0,"02 17, 2014",0077613252,This book was missing pages!!! Important pages...,missing pages!!
4,5.0,"10 14, 2013",0077775473,I have used LearnSmart and can officially say ...,Best study product out there!


In [4]:
### load the mega data

mega_data = []
with gzip.open('meta_Software.json.gz') as f:
    for l in f:
        mega_data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(mega_data))

# first row of the list
print(mega_data[1])

26790
{'category': [], 'tech1': '', 'description': ['', '<b>Latin rhythms that will get your kids singing in Spanish</b>', '<i>Sing, Watch, and Learn Spanish</i> helps your kids ages four through eight take a giant step in learning Spanish by combining two time-honored methods kids have always used to develop their language skills: Imitating other kids and singing along with simple melodies. This charming DVD contains 16 music videos featuring kids engaged in fun activities, from visiting animals at the zoo to comparing clothing sizes in grandmas closet.', 'Each video features an original song of authentic Latin rhythms that gets kids singing along with the children on screen. As they watch, listen, and sing along, kids absorb 300 Spanish words, each of which is shouted out in a song and displayed as a subtitle on screen.', '', '', ''], 'fit': '', 'title': 'Sing, Watch, &amp; Learn Spanish (DVD + Guide): 16 Fun Music Videos to Introduce Your Child to Spanish', 'also_buy': [], 'tech2': 

In [5]:
# convert list into pandas dataframe

df_product = pd.DataFrame.from_dict(mega_data)

#filter out the non-verified reviews, and leave only the columns we want
df_product = df_product[["asin", "rank"]]
df_product = df_product.rename(columns={"asin": "product_id"})
print(len(df_product))
df_product.head()
# TODO:filter out the products that has reviews, rank in software
# if(product_id)
# df_software = df_product[df_product['rank']]

26790


,product_id,rank
0,0030672120,"25,550 in Software ("
1,0071480935,"15,792 in Software ("
2,007329506X,"16,900 in Software ("
3,0073513458,"12,986 in Software ("
4,0073525758,"14,861 in Software ("


In [6]:
df_product = df_product.dropna(axis='index')
print(len(df_product))
df_product = df_product[df_product['rank'].str.contains('Software', na=False)]
print(len(df_product))


26790
21316


In [7]:
import string
def toInt(str_in):
    try:
        str_in=''.join(i for i in str_in if i.isdigit())
        output = str_in
    except:
        output = np.nan
        
    return output

In [8]:
df_product['rank'] = df_product['rank'].apply(toInt)
df_product.head()

,product_id,rank
0,0030672120,25550
1,0071480935,15792
2,007329506X,16900
3,0073513458,12986
4,0073525758,14861


In [9]:
all_products_Y = {set}

for index, product_id, rank in df_product.itertuples():
    all_products_Y.add(product_id)
# print(all_products_Y)
print(len(all_products_Y))

17142


In [10]:
options = all_products_Y
  
# selecting rows that product ids are included in the product dataframe
df_review = df_review[df_review['product_id'].isin(options)]

In [41]:
np.random.seed(32) # for reproducible results.
df_product_x1 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[0]
df_product_x2 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[1]
df_product_x3 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[2]
df_product_x4 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[3]
df_product_x5 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[4]
df_product_x6 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[5]
df_product_x7 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[6]
df_product_x8 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[7]
df_product_x9 = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[8]
#test set
df_product_y = np.array_split(df_product.reindex(np.random.permutation(df_product.index)),10)[9]
#training set
# df_product_x = df_product_x1
df_product_x = pd.concat([df_product_x1, df_product_x2, df_product_x3, df_product_x4, df_product_x5, df_product_x6, df_product_x7, df_product_x8, df_product_x9])
print(len(df_product), len(df_product_x), len(df_product_y), (len(df_product_x) + len(df_product_y)))
df_product.head()

21316 19185 2131 21316


,product_id,rank
0,0030672120,25550
1,0071480935,15792
2,007329506X,16900
3,0073513458,12986
4,0073525758,14861


In [109]:
all_products = {set}
product_rank = {}
y = []
a = 0
s = 0
d = 0
f = 0
g = 0
h = 0
j = 0
k = 0
l = 0
m = 0
for index, product_id, rank in df_product_x.itertuples():
    all_products.add(product_id)   
#     product_rank.update({product_id: rank})
    if int(rank)<1000:
        product_rank.update({product_id: 1})
        a+=1
    elif int(rank)<5000:
        product_rank.update({product_id: 2})
        s+=1
    elif int(rank)<10000:
        product_rank.update({product_id: 3})
        d+=1
    elif int(rank)<15000:
        product_rank.update({product_id: 4})
        f+=1
    elif int(rank)<20000:
        product_rank.update({product_id: 5})
        g+=1
    elif int(rank)<27000:
        product_rank.update({product_id: 6})
        h+=1
    elif int(rank)<35000:
        product_rank.update({product_id: 7})
        l+=1
    elif int(rank)<45000:
        product_rank.update({product_id: 8})
        j+=1
    elif int(rank)<60000:
        product_rank.update({product_id: 9})
        m+=1
    else:
        product_rank.update({product_id:10})
        k+=1
 
print(a,s,d,f,g,h,l,j,m,k)
# print(product_rank['0030672120'])

225 1516 2217 2157 1931 2437 1966 1994 2317 2425


In [110]:
ratingsPerItem = defaultdict(list)
reviewsPerItem = defaultdict(list)
summariesPerItem = defaultdict(list)

for index, rating, reviewTime, product_id, reviewText, summary in df_review.itertuples():
    if(product_id in all_products):
        ratingsPerItem[product_id].append(rating)
        reviewsPerItem[product_id].append(reviewText)
        summariesPerItem[product_id].append(summary)
print(len(all_products))
print(len(df_review))
df_review.head()

print(ratingsPerItem['0077613252'])
print(len(ratingsPerItem))
# print(s)
print(summariesPerItem['0077613252'])

11532
302290
[4.0, 4.0, 1.0, 3.0, 5.0, 4.0, 4.0, 3.0, 4.0, 5.0, 5.0, 1.0, 2.0, 1.0]
11530
['Material Great', 'Health', 'ARE YOU KIDING ME?', 'missing pages!!', 'Easy for studying', 'came in great', 'Four Stars', "I wouldn't waste money on the book", 'Four Stars', 'Five Stars', 'Five Stars', 'ACCESS CARD ONLY!', 'Two Stars', 'Missing pages/chapters']


In [111]:
# for d in reviewsPerItem.items():
#     print(d[1])
#   r = ''.join([c for c in d[1].lower() if not c in punctuation])
#   for w in r.split():
#     wordCount[w] += 1

In [112]:
Xtrain = []
ytrain = []


def feature(datum):
    Xtrain.append(datum[1])
    ytrain.append(int(product_rank[datum[0]]))
    feat = []
    feat.append(1)# offset
    feat.append(len(datum[1])) 
    rl = []
    sl = []
    for reviews in reviewsPerItem[datum[0]]:
        rl.append(len(str(reviews))) #length of reviews
    for summary in summariesPerItem[datum[0]]:
        sl.append(len(str(summary))) #length of summaries
    feat.append(sum(rl)/len(rl)) 
    feat.append(sum(sl)/len(sl))
    feat.append(sum(datum[1])/len(datum[1])) #average rating
#     feat.append()
    return feat

X = [feature(d) for d in ratingsPerItem.items()]
print(X)

[[1, 14, 123.35714285714286, 14.642857142857142, 3.2857142857142856], [1, 29, 1667.551724137931, 35.62068965517241, 4.137931034482759], [1, 38, 1099.842105263158, 33.63157894736842, 4.526315789473684], [1, 28, 1107.142857142857, 37.285714285714285, 4.321428571428571], [1, 118, 696.4237288135594, 26.872881355932204, 4.627118644067797], [1, 41, 396.9756097560976, 24.390243902439025, 4.512195121951219], [1, 13, 293.38461538461536, 31.0, 4.3076923076923075], [1, 11, 381.27272727272725, 13.636363636363637, 3.1818181818181817], [1, 44, 884.6818181818181, 30.681818181818183, 4.045454545454546], [1, 33, 806.030303030303, 37.24242424242424, 4.151515151515151], [1, 128, 678.1171875, 29.2734375, 3.9765625], [1, 12, 291.75, 33.25, 4.5], [1, 22, 881.2727272727273, 33.0, 3.227272727272727], [1, 18, 166.77777777777777, 15.555555555555555, 4.611111111111111], [1, 14, 398.2142857142857, 22.714285714285715, 3.857142857142857], [1, 11, 1130.1818181818182, 29.545454545454547, 3.5454545454545454], [1, 12, 

In [113]:
# mod = linear_model.LogisticRegression(solver = 'saga')
mod = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
mod.fit(X, ytrain) 

# pred = mod.predict(Xvalid)
# correct = pred == yvalid
# sum(correct) / len(correct)

Ridge(fit_intercept=False)

In [114]:
pred = mod.predict(X)
# print(pred[:100])

print(pred)
print(ytrain)
# sum(correct) / len(correct)

[5.77298295 6.70381989 6.0463006  ... 6.40254024 6.19222743 5.82599125]
[5, 3, 3, 4, 2, 3, 3, 3, 3, 5, 2, 3, 6, 2, 2, 3, 2, 2, 2, 3, 2, 2, 7, 6, 5, 4, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 3, 3, 3, 2, 1, 4, 3, 2, 4, 3, 2, 3, 3, 2, 2, 3, 4, 4, 2, 3, 5, 10, 2, 2, 5, 2, 5, 5, 4, 6, 10, 6, 4, 4, 6, 4, 3, 3, 2, 2, 4, 5, 2, 4, 3, 3, 3, 4, 3, 3, 6, 3, 7, 2, 4, 2, 2, 3, 2, 6, 8, 3, 6, 8, 1, 2, 1, 3, 10, 5, 3, 4, 8, 4, 3, 6, 7, 7, 5, 4, 6, 6, 2, 8, 5, 4, 4, 2, 5, 4, 6, 2, 6, 3, 9, 3, 2, 4, 6, 3, 4, 2, 4, 5, 6, 3, 6, 8, 6, 4, 7, 3, 8, 9, 4, 5, 8, 6, 5, 5, 4, 5, 4, 2, 7, 4, 6, 1, 4, 4, 5, 3, 3, 2, 10, 1, 4, 5, 3, 3, 4, 6, 5, 6, 3, 4, 6, 5, 3, 7, 2, 2, 8, 8, 6, 4, 10, 7, 8, 4, 3, 2, 6, 7, 8, 2, 2, 6, 5, 7, 4, 5, 8, 4, 1, 8, 4, 7, 4, 4, 5, 4, 6, 5, 8, 5, 4, 5, 4, 5, 6, 7, 6, 3, 4, 5, 4, 6, 7, 3, 3, 6, 5, 6, 7, 7, 3, 4, 9, 6, 9, 5, 4, 6, 6, 5, 4, 6, 9, 4, 8, 6, 10, 8, 2, 8, 4, 4, 5, 6, 3, 7, 4, 6, 6, 8, 6, 8, 6, 4, 3, 3, 7, 3, 3, 4, 3, 3, 4, 2, 4, 3, 3, 5, 4, 4, 4, 4, 8, 6, 3, 3, 9, 7, 5, 8, 7, 4, 3, 3, 4, 

In [115]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)
MSE(pred, ytrain)

6.0863985571871995

In [116]:
# print(Xtrain)
# print(ytrain)
print(len(Xtrain), len(ytrain))

11530 11530
